<a href="https://colab.research.google.com/github/RohanSohani30/projects/blob/main/Skin_cancer/skin_cancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reading of datasets

#### Here we are using CNN 

In [ ]:
import tensorflow as tf

In [ ]:
BATCH_SIZE = 64 # number of datapoints per obsevation in new dataset
IMAGE_SIZE1 = 600 # pixl size of images from raw data
IMAGE_SIZE2=450
CHANNELS=3 # RGB channel
EPOCHS=50 # number of epochs for NN model

In [ ]:
dataset=tf.keras.utils.image_dataset_from_directory("destination_folder",
    seed=123,
    shuffle=True,
    #image_size=(IMAGE_SIZE1,IMAGE_SIZE2),
    batch_size=BATCH_SIZE) # this is new dataset

Found 10015 files belonging to 7 classes.


In [ ]:
type(dataset)

tensorflow.python.data.ops.dataset_ops.BatchDataset

In [ ]:
class_names=dataset.class_names # Target variable
class_names

['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']

In [ ]:
d_pt=len(dataset) 
d_pt

157

In [ ]:
157*64

10048

In [ ]:
#Every Element in dataset is Batch of 32 images

In [ ]:
train_ds=dataset.take(int(d_pt*0.8))
train_ds

<TakeDataset element_spec=(TensorSpec(shape=(None, 600, 450, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [ ]:
len(train_ds)

125

In [ ]:
# took250 obsevations from new dataset as training data i.e 250*32 just like slicing operation

In [ ]:
test=dataset.skip(125)


In [ ]:
len(test)

32

In [ ]:
# skipped first 250 observation for test data

In [ ]:
32/2

16.0

In [ ]:
val_ds=test.take(16) # validation data is used for validation after evry epoch
test_ds=test.skip(16) # test data is used after all epochs are done for model testing
len(test_ds)

16

In [ ]:
# test data is divided into validation data and test data

In [ ]:
from tensorflow.keras import models, layers

In [ ]:
# Resizing and rescaling of input images
resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(256, 256),
  layers.experimental.preprocessing.Rescaling(1./255),
])

In [ ]:
# data augmentation of input images with flips and rotation

In [ ]:
data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.2),
])

In [ ]:
train_ds = train_ds.map(
    lambda x, y: (data_augmentation(x, training=True), y)
).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
# Model Building
#convolution layer=> flattend layer => dense layer 

In [ ]:
input_shape = (BATCH_SIZE, IMAGE_SIZE1, IMAGE_SIZE2, CHANNELS)
n_classes = 7

model = models.Sequential([
    resize_and_rescale,
    layers.Conv2D(120, kernel_size = (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])

model.build(input_shape=input_shape)

In [ ]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_4 (Sequential)   (64, 256, 256, 3)         0         
                                                                 
 conv2d_12 (Conv2D)          (64, 254, 254, 120)       3360      
                                                                 
 max_pooling2d_12 (MaxPoolin  (64, 127, 127, 120)      0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (64, 125, 125, 64)        69184     
                                                                 
 max_pooling2d_13 (MaxPoolin  (64, 62, 62, 64)         0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (64, 60, 60, 64)         

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
# for cpu runtime

In [ ]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
history = model.fit(
    train_ds,
    batch_size=BATCH_SIZE,
    validation_data=val_ds,
    verbose=1,
    epochs=5,
)

Epoch 1/5
157/157 [==============================] - 2155s 9s/step - loss: 1.0587 - accuracy: 0.6667
Epoch 2/5
157/157 [==============================] - 1491s 9s/step - loss: 0.9446 - accuracy: 0.6704
Epoch 3/5
157/157 [==============================] - 1471s 9s/step - loss: 0.9015 - accuracy: 0.6793
Epoch 4/5
157/157 [==============================] - 1305s 8s/step - loss: 0.8400 - accuracy: 0.6970
Epoch 5/5
157/157 [==============================] - 1334s 9s/step - loss: 0.8268 - accuracy: 0.6981


In [ ]:
type(test_ds)

tensorflow.python.data.ops.dataset_ops.PrefetchDataset

In [ ]:
model.evaluate(test_ds)

16/16 [==============================] - 73s 2s/step - loss: 1.9232 - accuracy: 0.6428


[1.9232177734375, 0.6427850723266602]

In [ ]:
 meta.dx.unique()

array(['bkl', 'nv', 'df', 'mel', 'vasc', 'bcc', 'akiec'], dtype=object)